<a href="https://colab.research.google.com/github/abhamedewar/Deep-Learning/blob/main/walk_with_stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
notebook by - Kartikeya Bhardwaj (twitter: @kartike_ya)
ported from - https://gist.github.com/nateraw/c989468b74c616ebbc6474aa8cdd9e53
"""

'\nnotebook by - Kartikeya Bhardwaj (twitter: @kartike_ya)\nported from - https://gist.github.com/nateraw/c989468b74c616ebbc6474aa8cdd9e53\n'

## Setup & Imports

You'll need to log into huggingface and accept the terms of the licence for this model - see the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4) for details. 

In [1]:
!pip install -q --upgrade transformers diffusers ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.0/852.0 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 413.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 5.2 MB/s eta 0:00:00


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from transformers import logging
from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler
from tqdm.auto import tqdm
from torch import autocast
from PIL import Image
from matplotlib import pyplot as plt
import numpy
from torchvision import transforms as tfms

# For video display:
from IPython.display import HTML
from base64 import b64encode

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

## Loading the models

This code (and that in the next section) comes from the [Huggingface example notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb). 

This will download and set up the relevant models and components we'll be using. Let's just run this for now and move on to the next section to check that it all works before diving deeper.

If you've loaded a pipeline, you can also access these components using `pipe.unet`, `pipe.vae` and so on.

In [3]:
# Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

# The noise scheduler
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

# To the GPU we go!
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device);

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


## Diffusion func

In [4]:
def diffuse(text_embeddings, init_noise):

  height = 512                        # default height of Stable Diffusion
  width = 512                         # default width of Stable Diffusion
  num_inference_steps = 30            # Number of denoising steps
  guidance_scale = 7.5                # Scale for classifier-free guidance
  generator = torch.manual_seed(42)   # Seed generator to create the inital latent noise
  batch_size = 1

  # Prep Scheduler
  scheduler.set_timesteps(num_inference_steps)

  # Prep latents
  latents = init_noise
  latents = latents.to(torch_device)
  latents = latents * scheduler.init_noise_sigma # Scaling (previous versions did latents = latents * self.scheduler.sigmas[0]

  # Loop
  with autocast("cuda"):
      for i, t in tqdm(enumerate(scheduler.timesteps)):
          # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
          latent_model_input = torch.cat([latents] * 2)
          sigma = scheduler.sigmas[i]
          # Scale the latents (preconditioning):
          # latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5) # Diffusers 0.3 and below
          latent_model_input = scheduler.scale_model_input(latent_model_input, t)

          # predict the noise residual
          with torch.no_grad():
              noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

          # perform guidance
          noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
          noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

          # compute the previous noisy sample x_t -> x_t-1
          # latents = scheduler.step(noise_pred, i, latents)["prev_sample"] # Diffusers 0.3 and below
          latents = scheduler.step(noise_pred, t, latents).prev_sample

  # scale and decode the image latents with vae
  latents = 1 / 0.18215 * latents
  with torch.no_grad():
      image = vae.decode(latents).sample

  # Display
  image = (image / 2 + 0.5).clamp(0, 1)
  image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
  images = (image * 255).round().astype("uint8")
  pil_images = [Image.fromarray(image) for image in images]
  return pil_images[0]

Spherical Linear Interpolation (SLERP)

In [5]:
def slerp(t, v0, v1, DOT_THRESHOLD=0.9995):
    """ helper function to spherically interpolate two arrays v1 v2 """

    if not isinstance(v0, np.ndarray):
        inputs_are_torch = True
        input_device = v0.device
        v0 = v0.cpu().numpy()
        v1 = v1.cpu().numpy()

    dot = np.sum(v0 * v1 / (np.linalg.norm(v0) * np.linalg.norm(v1)))
    if np.abs(dot) > DOT_THRESHOLD:
        v2 = (1 - t) * v0 + t * v1
    else:
        theta_0 = np.arccos(dot)
        sin_theta_0 = np.sin(theta_0)
        theta_t = theta_0 * t
        sin_theta_t = np.sin(theta_t)
        s0 = np.sin(theta_0 - theta_t) / sin_theta_0
        s1 = sin_theta_t / sin_theta_0
        v2 = s0 * v0 + s1 * v1

    if inputs_are_torch:
        v2 = torch.from_numpy(v2).to(input_device)

    return v2

In [6]:
def embed(prompt):
  text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
  with torch.no_grad():
      text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
  max_length = text_input.input_ids.shape[-1]
  uncond_input = tokenizer(
      [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
  )
  with torch.no_grad():
      uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0] 
  text_embeddings = torch.cat([uncond_embeddings, text_embeddings])
  return text_embeddings

In [15]:

def pil_to_latent(input_im):
    # Single image -> single latent in a batch (so size 1, 4, 64, 64)
    with torch.no_grad():
        latent = vae.encode(tfms.ToTensor()(input_im).unsqueeze(0).to(torch_device)*2-1) # Note scaling
    return 0.18215 * latent.latent_dist.sample()

Loop to interpolate between two text embeddings as well as their noise embeddings

In [21]:
import numpy as np
import os
# prompt_1 = "an oil paint sketch of a ford model t"
prompt_1 = "Boxing Practice start"
seed_1 = 42
height = 512
width = 512 
batch_size = 1
text_embed_1 = embed(prompt_1)


noise_1 = Image.open('/content/BoxingPractice-001.png').convert('RGB')
noise_1 = pil_to_latent(noise_1)

# noise_1  = torch.randn(
#   (batch_size, unet.in_channels, height // 8, width // 8),
#   generator=torch.manual_seed(seed_1),
# )

prompt_2 = " an oil paint sketch of a ferrari"
# prompt_2 = "Boxing Practice end"
seed_2 = 1000
text_embed_2 = embed(prompt_2)

noise_2 = Image.open('/content/BoxingPractice-001.png').convert('RGB')
noise_2 = pil_to_latent(noise_2)

# noise_2  = torch.randn(
#   (batch_size, unet.in_channels, height // 8, width // 8),
#   generator=torch.manual_seed(seed_2),
# )

num_steps = 5

for i, t in enumerate(np.linspace(0, 1, num_steps)):
  print("dreaming... ", i)

  cond_embedding = slerp(float(t), text_embed_1, text_embed_2)
  init = slerp(float(t), noise_1, noise_2)
  image = diffuse(cond_embedding, init)

  outpath = os.path.join("/content", 'BoxingPractice-%03d.png' % i)
  image.save(outpath)

dreaming...  0


0it [00:00, ?it/s]

dreaming...  1


0it [00:00, ?it/s]

dreaming...  2


0it [00:00, ?it/s]

dreaming...  3


0it [00:00, ?it/s]

dreaming...  4


0it [00:00, ?it/s]

Save Video

In [ ]:
!ffmpeg -r 10 -f image2 -s 512x512 -i /content/BoxingPractice-%03d.png -vcodec libx264 -crf 10 -pix_fmt yuv420p evolution_cars.mp4
